In [1]:
!pip install pyvi
!pip install tensorflow==1.14.0

     |████████████████████████████████| 8.5MB 4.2MB/s eta 0:00:01
     |████████████████████████████████| 440kB 45.3MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/40/fa/ee/3afb15958ad26f3aef88d61c316b4d1af8a97660aa24e6e6d7
Successfully built python-crfsuite


In [2]:
import os
#Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

#Importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.pooling import GlobalMaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.layers import *
from keras import backend
from sklearn.metrics import f1_score, confusion_matrix
import tensorflow as tf

from pyvi import ViTokenizer
from pyvi import ViUtils

Using TensorFlow backend.


### Importing the dataset
***

The dataset: 'imdb_master.csv' is read and loaded as pandas dataframe.  
Let's have a look at the data

# Importing the dataset


In [3]:
df2 = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/full_train.csv')
df2.head()

,Unnamed: 0,RevId,UserId,Comment,image_urls,Rating
0,0,3839333,10106093.0,"Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...",['https://images.foody.vn/res/g97/966781/s800/...,1.0
1,1,2824877,786914.0,Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,['https://images.foody.vn/res/g69/688413/s800/...,0.0
2,2,9816702,22467889.0,"Thời tiết lạnh như này, cả nhà rủ nhau đến leg...",['https://images.foody.vn/res/g72/715078/s800/...,1.0
3,3,2684585,1889449.0,Em có đọc review thấy mng bảo trà sữa nướng đề...,['https://images.foody.vn/res/g90/895545/s800/...,0.0
4,4,2737987,8839942.0,"Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",['https://images.foody.vn/res/g4/30186/s800/fo...,1.0


In [4]:
data_train2 = pd.DataFrame({'input':df2['Comment'],'label':df2['Rating']})
data_train2.label = data_train2.label.apply(lambda x: 1. if x == 1 else 0.)
data_train2 = data_train2.dropna()
data_train2 = data_train2.reset_index(drop=True)
X_train = list(data_train2['input'].values)
y_train = list(data_train2['label'].values)

### Text Preprocessing
***
Preprocessing the text so as to have a better data for our model.  
It comprises of steps such as removing non-ASCII characters, removing HTML tags, converting to lower-case, lemmatizing.

In [5]:
#Function for Text Preprocessing
def clean_text(X,y):
    idx = 0
    y_train = []
    processed = []
    for text in X:
        text = list(tf.keras.preprocessing.text.text_to_word_sequence(text))
        text = " ".join(text)
        input_text_pre_no_accent = str(ViUtils.remove_accents(text).decode("utf-8"))
        input_text_pre_accent = ViTokenizer.tokenize(text)
        input_text_pre_no_accent = ViTokenizer.tokenize(input_text_pre_no_accent)
        processed.append(input_text_pre_accent)
        processed.append(input_text_pre_no_accent)
        y_train.append(y[idx])
        y_train.append(y[idx])
        idx += 1
    return processed,y_train

Preprocessing the Training Set and Test set

In [6]:
X_train_final,y_train = clean_text(X_train,y_train)

### Attention Layer
***

The basic concept of attention is that not all words contribute equally to the meaning of a sentence. Hence, their contribution must be weighted.  
How attention works is, it basically extracts words that are important to the meaning of the sentence and aggregate the representation of those informative words to form a sentence vector.

In [7]:
# Attention Layer
class AttentionWithContext(Layer):

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

Some Useful Variables  


In [8]:
#Tokenization and Padding
vocab_size = 60000
maxlen = 300
encode_dim = 20
batch_size = 32
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_final)
tokenized_word_list = tokenizer.texts_to_sequences(X_train_final)
X_train_padded = pad_sequences(tokenized_word_list, maxlen = maxlen, padding='post')

In [9]:
#ModelCheckpoint
checkpoint = ModelCheckpoint('sentiment_classifier.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)

### Building the Model
***
The model used comprises of CuDNNLSTM with Attention layer on top of it, followed by a dense layer and finally a dense layer with sigmoid activation function to get the sentiment or the class.  
Optimiser used is ADAM

In [10]:
# Building the model
model = Sequential()
embed = Embedding(input_dim = vocab_size, output_dim = 32, input_length = X_train_padded.shape[1], dropout = 0.4) 
model.add(embed)
model.add(Bidirectional(CuDNNLSTM(256, return_sequences = True)))
model.add(Dropout(0.5))
model.add(AttentionWithContext())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(ReLU())
model.add(Dense(256))
model.add(ReLU())
model.add(Dense(1, activation = 'sigmoid'))
from keras.optimizers import adam
optim =  adam(lr=1e-4)
model.compile(loss = 'binary_crossentropy', 
              optimizer = optim, 
              metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 32)           1920000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 512)          593920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 512)          0         
_________________________________________________________________
attention_with_context_1 (At (None, 512)               263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 512)               0         
__________

### Training
***
Splitting the Training set into Training set and Validation set

In [11]:
from sklearn.model_selection import train_test_split
X_train_final2, X_val, y_train_final2, y_val = train_test_split(X_train_padded, y_train, test_size = 0.1)

In [12]:
#class weight
weight_for_0 = (1 / 0.27)#*((len(y))/2.0 )
weight_for_1 = (1 / 0.73)#*((len(y))/2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

In [13]:
checkpoint = ModelCheckpoint('sentiment_classifier.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only = True)

In [14]:
#Fitting the model
batch_size= 256

model.fit(X_train_final2, y_train_final2, 
          epochs = 35, batch_size = batch_size, verbose = 1,
          validation_data = [X_val, y_val],
          callbacks = [checkpoint],class_weight=class_weight)

Train on 16327 samples, validate on 1815 samples
Epoch 1/35
16327/16327 [==============================] - 14s 869us/step - loss: 1.2754 - acc: 0.7878 - val_loss: 0.6173 - val_acc: 0.7846

Epoch 00001: val_acc improved from -inf to 0.78457, saving model to sentiment_classifier.h5
Epoch 2/35
16327/16327 [==============================] - 8s 505us/step - loss: 1.2703 - acc: 0.7881 - val_loss: 0.6174 - val_acc: 0.7846

Epoch 00002: val_acc did not improve from 0.78457
Epoch 3/35
16327/16327 [==============================] - 8s 504us/step - loss: 1.2688 - acc: 0.7881 - val_loss: 0.5936 - val_acc: 0.7846

Epoch 00003: val_acc did not improve from 0.78457
Epoch 4/35
16327/16327 [==============================] - 8s 501us/step - loss: 1.1646 - acc: 0.7941 - val_loss: 0.5097 - val_acc: 0.7736

Epoch 00004: val_acc did not improve from 0.78457
Epoch 5/35
16327/16327 [==============================] - 8s 502us/step - loss: 0.7101 - acc: 0.8732 - val_loss: 0.3382 - val_acc: 0.8727

Epoch 00005: 

KeyboardInterrupt: 

### Testing
***
Converting the test data into sequences of integers and padding them.  
Loading the best model and calculating the accuracy

In [15]:
df = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/test.csv')
data_test = pd.DataFrame({'input':df['Comment'],'id':df["RevId"]})
X_test = data_test['input'].values

def clean_text_test(X):
    processed = []
    for text in X:
        text = list(tf.keras.preprocessing.text.text_to_word_sequence(str(text)))
        text = " ".join(text)
        input_text_pre_no_accent = str(ViUtils.remove_accents(text).decode("utf-8"))
        input_text_pre_accent = ViTokenizer.tokenize(text)
        processed.append(input_text_pre_accent)
    return processed

X_test_final = clean_text_test(X_test)

tokenized_word_list = tokenizer.texts_to_sequences(X_test_final)
X_test_padded = pad_sequences(tokenized_word_list, maxlen = maxlen, padding='post')
y_pred = model.predict(X_test_padded)

In [16]:
from keras.models import load_model
modelload = load_model('sentiment_classifier.h5', custom_objects = {"AttentionWithContext" : AttentionWithContext, "backend" : backend})
y_pred = modelload.predict(X_test_padded)

In [18]:
my_submission = pd.DataFrame({'RevId': np.array(df["RevId"]).reshape(5103), 'Rating': np.array(y_pred).reshape(5103)})
my_submission.to_csv('submitTrain.csv', index=False)